<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

###Load and clean the data (or determine the best method to drop observations when running tests)

In [1]:
#get data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2022-08-22 13:38:14--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.1s    

2022-08-22 13:38:15 (138 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [10]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

column_names = ['class-name', 'handicapped-infants', 'water-project-cost-sharing', 
 'adoption-of-the-budget-resolution', 'physician-fee-freeze', 'el-salvador-aid', 
 'religious-groups-in-schools', 'anti-satellite-test-ban', 'aid-to-nicaraguan-contras', 
 'mx-missile', 'immigration', 'synfuels-corporation-cutback', 
 'education-spending', 'superfund-right-to-sue', 'crime', 'duty-free-exports', 
 'export-administration-act-south-africa']

df=pd.read_csv('/content/house-votes-84.data', on_bad_lines='skip', 
                      header=None, names=column_names)
df.head()

,class-name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [13]:
#recode y/n to 
df=df.replace({'y':1, 'n':0, '?':None})
df.head()

,class-name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


###Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01

In [21]:
#lets look at adoption-of-the-budget-resolution
dem=df[df['class-name']=='democrat']['adoption-of-the-budget-resolution']
rep=df[df['class-name']=='republican']['adoption-of-the-budget-resolution']

print(len(dem))
print(len(rep))

267
168


In [22]:
print(dem.mean())
print(rep.mean())

0.8884615384615384
0.13414634146341464


In [18]:
#run ttest
ttest_ind(dem, rep, nan_policy='omit').pvalue<0.01 

True

###Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01

In [23]:
#lets look at religious-groups-in-schools
dem1=df[df['class-name']=='democrat']['religious-groups-in-schools']
rep1=df[df['class-name']=='republican']['religious-groups-in-schools']

print(len(dem))
print(len(rep))

267
168


In [24]:
print(dem1.mean())
print(rep1.mean())

0.47674418604651164
0.8975903614457831


In [25]:
#run ttest
ttest_ind(dem1, rep1, nan_policy='omit').pvalue<0.01 

True

###Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

In [34]:
#lets look at anti-satellite-test-ban
dem2=df[df['class-name']=='democrat']['water-project-cost-sharing']
rep2=df[df['class-name']=='republican']['water-project-cost-sharing']

In [35]:
print(dem2.mean())
print(rep2.mean())

0.502092050209205
0.5067567567567568


In [37]:
#run ttest
ttest_ind(dem2, rep2, nan_policy='omit').pvalue>0.01 

True

###STRECH
- Refactor your code into functions so it's easy to rerun with arbitrary variables
- Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [ ]:
#for this dataset lets write a function 
# that would get data and a column as an input 
# and return if there is sifgnificant diff between the groups 
# on that column

def is_there_diff(df, col=str):

  dem=df[df['class-name']=='democrat'][col]
  rep=df[df['class-name']=='republican'][col]

  

